# iX Mobile Banking Prediction Challenge

PROBLEM IDENTIFICATION

Banks and financial service providers value knowing what habits their clients follow. This allows them to tailor products and services. This challenge asked to build a machine learning model to predict if individuals across Africa and around the world use mobile or internet banking.

This solution will provide insight into people’s financial behavior, which can help financial services providers, including insurance companies and banks, tailor the services they provide their clients.

IMPORTING LIBRARIES

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import warnings
warnings.simplefilter("ignore")
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling as pp
import os

#machine learning
from sklearn.preprocessing import *
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score 
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [26]:
# Get titanic & test csv files as a DataFrame
Train_df = pd.read_csv("Train.csv")
Test_df = pd.read_csv("Test.csv")
Submission = pd.read_csv("SampleSubmission.csv")
variableDesciprition = pd.read_csv("VariableDefinitions.csv")

# Preview the data
Train_df.head()

,ID,country_code,region,age,FQ1,FQ2,FQ3,FQ4,FQ5,FQ6,...,FQ27,FQ28,FQ29,FQ30,FQ31,FQ32,FQ33,FQ34,FQ37,Target
0,ID_000J8GTZ,1,6,35.0,2,NaN,NaN,2,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,0,0
1,ID_000QLXZM,32,7,70.0,2,NaN,NaN,2,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,1.0,2.0,0,0
2,ID_001728I2,71,7,22.0,2,1.0,NaN,2,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,2.0,1.0,1,0
3,ID_001R7IDN,48,3,27.0,1,NaN,NaN,2,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,1,0
4,ID_0029QKF8,25,0,79.0,2,NaN,NaN,2,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,2.0,1.0,1.0,1,0


In [27]:
variableDesciprition

,ID,Person unique/random identification
0,country_code,The country codes
1,region,"Region_codes of a given country,.i.e., South A..."
2,age,Individual age
3,FQ1,Has ATM/debit card
4,FQ2,Is the ATM connected to an account with your n...
5,FQ3,Purchased with the ATM
6,FQ4,Has a credit card
7,FQ5,Used the credit card in 12 months
8,FQ6,Money deposits to a bank account in the last 1...
9,FQ7,Money withdrawals from a bank account in the l...


In [28]:
Train_df.info()
print('_'*40)
Test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108446 entries, 0 to 108445
Data columns (total 42 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            108446 non-null  object 
 1   country_code  108446 non-null  int64  
 2   region        108446 non-null  int64  
 3   age           108124 non-null  float64
 4   FQ1           108446 non-null  int64  
 5   FQ2           49124 non-null   float64
 6   FQ3           46218 non-null   float64
 7   FQ4           108446 non-null  int64  
 8   FQ5           21185 non-null   float64
 9   FQ6           60659 non-null   float64
 10  FQ7           60620 non-null   float64
 11  FQ8           108446 non-null  int64  
 12  FQ9           108446 non-null  int64  
 13  FQ10          108446 non-null  int64  
 14  FQ11          83876 non-null   float64
 15  FQ12          108446 non-null  int64  
 16  FQ13          108446 non-null  int64  
 17  FQ14          108446 non-null  int64  
 18  FQ15

In [29]:
Train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
country_code,108446.0,68.544953,41.529264,0.0,33.0,65.0,105.0,143.0
region,108446.0,2.894242,2.286505,-1.0,1.0,3.0,4.0,7.0
age,108124.0,41.857395,17.876105,15.0,27.0,39.0,55.0,99.0
FQ1,108446.0,1.563294,0.530077,1.0,1.0,2.0,2.0,4.0
FQ2,49124.0,1.063716,0.288075,1.0,1.0,1.0,1.0,4.0
FQ3,46218.0,1.299710,0.468503,1.0,1.0,1.0,2.0,4.0
FQ4,108446.0,1.824622,0.435942,1.0,2.0,2.0,2.0,4.0
FQ5,21185.0,1.160113,0.383827,1.0,1.0,1.0,1.0,4.0
FQ6,60659.0,1.223907,0.450140,1.0,1.0,1.0,1.0,4.0
FQ7,60620.0,1.206961,0.440780,1.0,1.0,1.0,1.0,4.0


TARGET DISTRIBUTION

In [30]:
Target_counts = Train_df.groupby('Target').size()
print(Target_counts)

Target
0    78735
1    29711
dtype: int64


From the Target distribution it indicates that 78735 people are not using mobile nor internet banking while 29711 people(customers) use mobile or internet banking

CHECKING CORRELATION OF DATA

In [31]:
Train_df.corr()

,country_code,region,age,FQ1,FQ2,FQ3,FQ4,FQ5,FQ6,FQ7,...,FQ27,FQ28,FQ29,FQ30,FQ31,FQ32,FQ33,FQ34,FQ37,Target
country_code,1.000000,0.002045,0.003993,0.001961,-0.001755,0.006650,-0.004848,0.007184,-0.003313,0.001120,...,0.005027,0.016163,-0.002020,0.009163,0.046136,0.002970,0.002925,0.005749,0.000470,0.000593
region,0.002045,1.000000,-0.000828,0.002407,0.001205,0.000201,-0.000254,0.010574,0.004886,-0.000531,...,0.017660,0.001233,-0.000858,-0.031676,-0.032487,0.001703,0.004678,0.002259,0.000411,0.002787
age,0.003993,-0.000828,1.000000,0.001280,0.003393,0.004072,0.005115,-0.003840,0.002942,-0.003869,...,0.000869,0.013508,-0.001169,0.000717,-0.027512,-0.002868,0.000793,-0.001288,0.002975,-0.001732
FQ1,0.001961,0.002407,0.001280,1.000000,0.004941,0.003003,0.000291,0.012897,0.007370,-0.000162,...,0.030461,0.042179,0.006813,0.035120,-0.021440,0.000432,-0.001298,0.002763,0.001269,-0.004409
FQ2,-0.001755,0.001205,0.003393,0.004941,1.000000,-0.002296,-0.001996,-0.014220,-0.007442,-0.000149,...,-0.040401,-0.032575,-0.000469,0.004976,0.055057,0.000550,0.002175,-0.001690,-0.000626,0.004199
FQ3,0.006650,0.000201,0.004072,0.003003,-0.002296,1.000000,0.002398,0.006970,0.010562,0.005371,...,0.024474,-0.038157,-0.011934,-0.036051,0.009167,0.016962,0.006983,-0.006219,-0.004863,0.002512
FQ4,-0.004848,-0.000254,0.005115,0.000291,-0.001996,0.002398,1.000000,0.005662,-0.001392,0.000070,...,0.024687,-0.008713,0.002822,0.008543,-0.016221,0.003057,0.002298,0.004235,-0.002425,0.000410
FQ5,0.007184,0.010574,-0.003840,0.012897,-0.014220,0.006970,0.005662,1.000000,-0.014271,-0.002153,...,-0.063995,-0.046123,0.011249,0.065046,0.096196,0.000598,0.000265,-0.004504,0.001224,0.014263
FQ6,-0.003313,0.004886,0.002942,0.007370,-0.007442,0.010562,-0.001392,-0.014271,1.000000,0.001077,...,-0.026663,0.024191,-0.004828,0.028170,-0.071173,-0.003467,0.001334,0.001133,0.000321,0.004330
FQ7,0.001120,-0.000531,-0.003869,-0.000162,-0.000149,0.005371,0.000070,-0.002153,0.001077,1.000000,...,-0.005145,-0.013474,0.003974,0.027392,-0.077646,-0.001541,-0.002905,-0.000398,-0.002455,-0.002377


CHEKING SKEWNESS

In [32]:
Train_df.agg(['skew']).transpose()

,skew
country_code,0.115109
region,0.213913
age,0.487306
FQ1,0.232395
FQ2,5.490947
FQ3,1.075769
FQ4,-0.558380
FQ5,2.319267
FQ6,2.016291
FQ7,2.218834


From the above it seem that data are positvely and negatively skewed, thus transformation must be done

PANDAS PROFILING

In [33]:
#profile = pp.ProfileReport(Train_df, title = "iX Mobile Banking Prediction")
#profile

# PRE-PROCESSING OF DATA

CHECKING FOR NULL VALUES

In [34]:
Train_df.isnull().sum().any()

True

CHEKING NULL VALUES IN %

In [35]:
percent = Train_df.isnull().sum()*100/len(Train_df)
missing_Train =pd.DataFrame({'missing': percent})
missing_Train.sort_values('missing', ascending=False)

,missing
FQ31,99.198680
FQ28,98.611290
FQ30,98.049721
FQ27,97.049223
FQ17,89.536728
FQ36,89.411320
FQ5,80.464932
FQ35,76.127289
FQ24,64.561164
FQ3,57.381554


Filling the null values

In [36]:
for i in ['age','FQ2','FQ3','FQ5','FQ6','FQ7','FQ11','FQ17','FQ19','FQ20','FQ21','FQ24','FQ27','FQ28',
          'FQ29','FQ30','FQ31','FQ32','FQ33','FQ34','FQ35','FQ36']:
    Train_df[i] = Train_df[i].fillna(value=0.0)
    Test_df[i] = Test_df[i].fillna(value=0.0)

Remove Duplicate Rows from Train data if present

In [37]:
#  Removes Data Duplicates while Retaining the First one - Similar to SQL DISTINCT :

def remove_duplicate(Train_df):
    
    print("BEFORE REMOVING DUPLICATES - No. of Rows = ",Train_df.shape[0])
    Train_df.drop_duplicates(keep="first", inplace=True) 
    print("AFTER REMOVING DUPLICATES  - No. of Rows = ",Train_df.shape[0])
    
    return Train_df

# Remove Duplicates from "train" data :

train = remove_duplicate(Train_df)

# No Duplicates at all !!!

BEFORE REMOVING DUPLICATES - No. of Rows =  108446
AFTER REMOVING DUPLICATES  - No. of Rows =  108446


DATA TRANSFORMATION

# FEATURE SELECTION

Create the X and y variables for the  dataset

In [38]:
predictor_train_x = Train_df.drop(['ID','Target'], axis=1)
predictor_train_y= Train_df.pop('Target')
predictor_test = Test_df.drop("ID",axis=1)

# Scaling

In [39]:
def data_scaling( scaling_strategy , scaling_data , scaling_columns ):
    
    if    scaling_strategy =="RobustScaler" :
        scaling_data[scaling_columns] = RobustScaler().fit_transform(scaling_data[scaling_columns])
        
    elif  scaling_strategy =="StandardScaler" :
        scaling_data[scaling_columns] = StandardScaler().fit_transform(scaling_data[scaling_columns])
        
    elif  scaling_strategy =="MinMaxScaler" :
        scaling_data[scaling_columns] = MinMaxScaler().fit_transform(scaling_data[scaling_columns])
        
    elif  scaling_strategy =="MaxAbsScaler" :
        scaling_data[scaling_columns] = MaxAbsScaler().fit_transform(scaling_data[scaling_columns])
        
    else :  # If any other scaling send by mistake still perform Robust Scalar
        scaling_data[scaling_columns] = RobustScaler().fit_transform(scaling_data[scaling_columns])
    
    return scaling_data

scaling_strategy = ["RobustScaler", "StandardScaler","MinMaxScaler","MaxAbsScaler"]

predictor_train_scale = data_scaling( scaling_strategy[1] , predictor_train_x, predictor_train_x.columns )
predictor_test_scale = data_scaling( scaling_strategy [1] , predictor_test , predictor_test.columns )
predictor_train_scale = predictor_train_x
predictor_test_scale = predictor_test

In [41]:
print("predictor_train_encode SHAPE   : ",predictor_train_scale.shape)
display("predictor_train_encode COLUMNS : ",predictor_train_scale.head())

print("predictor_test_encode SHAPE   : ",predictor_test_scale.shape)
display("predictor_test_encode COLUMNS : ",predictor_test_scale.head())

predictor_train_encode SHAPE   :  (108446, 40)


'predictor_train_encode COLUMNS : '

,country_code,region,age,FQ1,FQ2,FQ3,FQ4,FQ5,FQ6,FQ7,...,FQ26,FQ27,FQ28,FQ29,FQ30,FQ31,FQ32,FQ33,FQ34,FQ37
0,-1.626450,1.358305,-0.374183,0.823857,-0.854512,-0.778197,0.402297,-0.462284,-0.985508,0.475687,...,0.321763,-0.163931,-0.111369,-0.518241,-0.133402,-0.083749,-1.073746,-0.447436,0.343052,-1.308965
1,-0.879985,1.795656,1.570892,0.823857,-0.854512,-0.778197,0.402297,-0.462284,-0.985508,0.475687,...,-2.411585,-0.163931,-0.111369,0.661019,-0.133402,-0.083749,-1.073746,-0.447436,2.032543,-1.308965
2,0.059116,1.795656,-1.096639,0.823857,0.918910,-0.778197,0.402297,-0.462284,-0.985508,0.475687,...,-2.411585,-0.163931,-0.111369,0.661019,-0.133402,-0.083749,-1.073746,2.059803,0.343052,0.763963
3,-0.494713,0.046253,-0.818771,-1.062670,-0.854512,-0.778197,0.402297,-0.462284,1.893612,-0.986511,...,0.321763,-0.163931,-0.111369,-1.697500,-0.133402,-0.083749,0.991569,-0.447436,0.343052,0.763963
4,-1.048541,-1.265799,2.071054,0.823857,-0.854512,-0.778197,0.402297,-0.462284,-0.985508,-0.986511,...,0.321763,-0.163931,-0.111369,0.661019,-0.133402,-0.083749,0.991569,-0.447436,0.343052,0.763963


predictor_test_encode SHAPE   :  (46477, 40)


'predictor_test_encode COLUMNS : '

,country_code,region,age,FQ1,FQ2,FQ3,FQ4,FQ5,FQ6,FQ7,...,FQ26,FQ27,FQ28,FQ29,FQ30,FQ31,FQ32,FQ33,FQ34,FQ37
0,-0.707413,-0.396012,-1.087497,0.830807,-0.846988,-0.782009,0.402233,-0.458190,0.460522,-0.984873,...,0.325762,-0.1682,-0.109844,-1.688103,-0.131091,-0.082233,0.997626,-0.455402,0.328956,-1.308467
1,-0.924232,-0.396012,1.121718,-1.061729,-0.846988,-0.782009,0.402233,-0.458190,0.460522,0.478863,...,0.325762,-0.1682,-0.109844,0.667886,-0.131091,7.367980,-0.036513,-0.455402,0.328956,-1.308467
2,-0.081047,0.480104,-0.369502,0.830807,0.934577,-0.782009,-1.890176,1.605907,-0.981581,0.478863,...,0.325762,-0.1682,-0.109844,-1.688103,-0.131091,-0.082233,-1.070652,-0.455402,-1.360239,-1.308467
3,1.316229,-1.272127,-0.977036,0.830807,0.934577,-0.782009,0.402233,-0.458190,0.460522,0.478863,...,0.325762,-0.1682,-0.109844,0.667886,-0.131091,-0.082233,-1.070652,-0.455402,0.328956,0.764253
4,-0.032865,-0.396012,-0.921806,0.830807,-0.846988,-0.782009,-1.890176,-0.458190,-0.981581,1.942598,...,0.325762,-0.1682,-0.109844,-0.510109,-0.131091,-0.082233,-1.070652,2.052086,0.328956,0.764253


In [42]:
import re
predictor_train_scale = predictor_train_scale.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
predictor_test_scale = predictor_test_scale.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [46]:
!pip install lightgbm

     |████████████████████████████████| 2.0 MB 852 kB/s eta 0:00:01


In [47]:
# Boosting Algorithm Libraries :

from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# Model Evaluation Metric & Cross Validation Libraries :
from sklearn.metrics import roc_auc_score, auc, roc_curve

In [55]:
kf=KFold(n_splits=10,shuffle=True)

preds_1   = list()
y_pred_1  = []
rocauc_score = []

for i,(train_idx,val_idx) in enumerate(kf.split(predictor_train_scale)):    
    
    X_train, y_train = predictor_train_scale.iloc[train_idx,:],predictor_train_y.iloc[train_idx]    
    X_val, y_val = predictor_train_scale.iloc[val_idx, :], predictor_train_y.iloc[val_idx]
   
    print('\nFold: {}\n'.format(i+1))

    lg= LGBMClassifier(      metric = 'auc',                
                       # GPU PARAMETERS #
                       #device = "gpu",
                       #gpu_device_id =0,
                       #max_bin = 63,
                       #gpu_platform_id=1,
                       # GPU PARAMETERS #
                       
                       n_estimators=50000,    
                       bagging_fraction=0.95, 
                       subsample_freq = 2, 
                       objective ="binary",
                       min_samples_leaf= 2,
                       importance_type = "gain",
                       verbosity = -1,
                       random_state=294,
                       num_leaves = 300,
                       boosting_type = 'gbdt',
                       learning_rate=0.15,
                       max_depth=4, 
                       
                        n_jobs=-1 
                      )

    lg.fit(X_train, y_train
          ,eval_set=[(X_train, y_train),(X_val, y_val)]           
          ,early_stopping_rounds=100
          ,verbose=100
          )

    roc_auc = roc_auc_score(y_val,lg.predict_proba(X_val)[:, 1])
    rocauc_score.append(roc_auc)
    preds_1.append(lg.predict_proba(predictor_test_scale[predictor_test_scale.columns])[:, 1])
    
y_pred_final_1 = np.mean(preds_1, axis=0)    
Submission['Target']=y_pred_final_1

Blend_model_1 = Submission.copy()


Fold: 1

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.597776	valid_1's auc: 0.49918
Early stopping, best iteration is:
[5]	training's auc: 0.516628	valid_1's auc: 0.505953

Fold: 2

[LightGBM] [Warning] Unknown parameter: min_samples_leaf
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.599359	valid_1's auc: 0.492378
Early stopping, best iteration is:
[2]	training's auc: 0.509908	valid_1's auc: 0.503862

Fold: 3

[LightGBM] [Warning] Unknown parameter: min_samples_leaf
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.599949	valid_1's auc: 0.497272
[200]	training's auc: 0.630465	valid_1's auc: 0.495316
Early stopping, best iteration is:
[1

In [56]:
print('ROC_AUC - CV: {}'.format((sum(rocauc_score)/10)),'\n')
print("Score : ",rocauc_score)

ROC_AUC - CV: 0.5061885761253947 

Score :  [0.5059534074058154, 0.5038616786706441, 0.49854144407730017, 0.5091699826990369, 0.5023810590633463, 0.5073727762831896, 0.5109175441708624, 0.5117442844151721, 0.5062482429253445, 0.5056953415432355]


In [57]:
display("sample_submmission",Submission)
sub_file_name_1 = "Beginner_Submission.csv"
Submission.to_csv(sub_file_name_1,index=False)
Submission.head(5)

'sample_submmission'

,ID,Target
0,ID_000YI58E,0.277209
1,ID_001SP4JF,0.268240
2,ID_001VOF6S,0.273724
3,ID_0030LULG,0.273870
4,ID_0037PZ3R,0.278372
...,...,...
46472,ID_ZZYOTVBJ,0.345966
46473,ID_ZZYSX122,0.277904
46474,ID_ZZYXQDSD,0.273102
46475,ID_ZZZH9SS4,0.278900


,ID,Target
0,ID_000YI58E,0.277209
1,ID_001SP4JF,0.268240
2,ID_001VOF6S,0.273724
3,ID_0030LULG,0.273870
4,ID_0037PZ3R,0.278372


In [40]:
# initilize the model
#model = ExtraTreesClassifier()

# fit the model to be trained
#model.fit(X_Train,y_Train)

#use inbuilt class feature_importances of tree based classifiers
#print(model.feature_importances_) 

#plot graph of feature importances for better visualization
#feat_importances = pd.Series(model.feature_importances_, index=X_Train.columns)
#feat_importances.nlargest(7).plot(kind='barh')
#plt.show()

NameError: name 'p' is not defined

# Evaluate the Performance of Machine Learning Algorithms

In [ ]:
#num_folds = 10
#seed = 7777
#kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
#model = LogisticRegression().fit(X_Train,y_Train)
#Y_pred = model.predict(X_test)
#results = cross_val_score(model, X_Train, y_Train, cv=kfold) 
#print(("Accuracy: %.3f%% (%.3f%%)") % (results.mean()*100.0, results.std()*100.0))


# Machine Learning Algorithm Performance Metrics

In [ ]:
#scoring = 'accuracy'
#results = cross_val_score(model, X_Train, y_Train, cv=kfold, scoring=scoring)
#print(("Accuracy: %.3f (%.3f)") % (results.mean()*100.0, results.std()*100.0))

# Algorithm Spot-Checking

In [ ]:
#model= model = LinearDiscriminantAnalysis()
#results = cross_val_score(model, X_Train, y_Train, cv=kfold)
#print(results.mean())

In [ ]:
#model = KNeighborsClassifier()
#results = cross_val_score(model, X_Train, y_Train, cv=kfold)
#print(results.mean())

# Improve Performance with Ensembles

In [ ]:
#cart = DecisionTreeClassifier()
#num_trees = 100
#model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
#results = cross_val_score(model, X_Train, y_Train, cv=kfold)
#print(results.mean())

In [ ]:
#submission = pd.DataFrame({
 #       "ID": X_test["ID"],
  #      "Target": Y_pred
   # })
#submission.to_csv('My_First_Submission.csv', index=False)

In [ ]:
submission.head()